## 1) 問題分析結果

### 競技プログラミング視点（最速・最小メモリ）

* **狙い**: 与えられた直角三角形の一辺（cathetus）`a` から、整数の **ピタゴラス数** `(a, b, c)` を **O(1)** で構成する。
* **結論**: `a` の **偶奇**で閉形式（公式）を使えば探索不要。

### 業務開発視点（可読性・保守性・型安全）

* **入力検証**（型/範囲）を分離し、メインロジックは純粋関数化。
* 例外（`ValueError`, `TypeError`）を明確化して、利用側で握れるようにする。

### Python（CPython 3.13.3）特有の考慮

* `a*a` は Python の任意精度整数で安全。剰余や除算は **整数演算（`//`）**で完結。
* ループや探索を排して **算術のみ**にするのが最速・最堅牢（分岐1回 + 乗算数回）。

---

## 2) アルゴリズム比較表

| アプローチ                    |      時間計算量 |    空間計算量 | Python実装コスト | 可読性 | 標準ライブラリ活用    | CPython最適化 | 備考            |
| ------------------------ | ---------: | -------: | ----------: | --: | ------------ | ---------- | ------------- |
| 方法A: 偶奇で閉形式（構成公式）        |   **O(1)** | **O(1)** |           低 | ★★★ | 不要           | **適**      | どれか1解で良い要件に最適 |
| 方法B: `b` を探索して `c` を平方判定 | O(a)〜O(a²) |     O(1) |           中 | ★★☆ | `math.isqrt` | 不適         | 制約が大きいとTLEリスク |

---

## 3) 採用アルゴリズムと根拠

### 選択理由

* 要件は「解が複数ありうるが **どれか1つ**」でよい → **生成式が最適解**。
* 偶数・奇数それぞれで確実に整数解が作れる。

### 具体式（`a` が直角の一辺）

* **`a` が奇数**:

  * `b = (a^2 - 1) / 2`
  * `c = (a^2 + 1) / 2`
* **`a` が偶数**（`a = 2k`）:

  * `b = k^2 - 1`
  * `c = k^2 + 1`

---

## 4) 実装パターン（2パターン提供）

* `solve_production`: 検証・例外込み（業務品質）
* `solve_competitive`: 最短・最速（競プロ品質）
* HackerRank提出では **指定関数 `pythagoreanTriple(a)`** を実装（I/O 直接使用なし）

---

## 5) 検証観点（設計レベル）

* 境界: `a` が奇数最小ケース、偶数最小ケース（問題の制約が通常 `a > 2` を想定）
* 大入力: `a` が非常に大きい場合でも **O(1)** で完走
* 型: `a` が `int` 以外なら `TypeError`（production側）

---

```python
#!/bin/python3

import math
import os
import random
import re
import sys
from typing import Any, List

#
# Complete the 'pythagoreanTriple' function below.
#
# The function is expected to return an INTEGER_ARRAY.
# The function accepts INTEGER a as parameter.
#


def solve_production(a: int) -> List[int]:
    """
    業務開発向け実装（型安全・エラーハンドリング重視）

    Args:
        a: 直角三角形の一辺（cathetus）の長さ（正の整数）
    Returns:
        [a, b, c] を返す（順序は任意）
    Raises:
        TypeError: a が int でない
        ValueError: a が制約（通常 a > 2）を満たさない
    """
    _validate_input(a)
    return _main_algorithm(a)


def solve_competitive(a: int) -> List[int]:
    """
    競技プログラミング向け実装（性能最優先）

    Time Complexity: O(1)
    Space Complexity: O(1)
    """
    return _main_algorithm(a)


def _validate_input(a: Any) -> None:
    if not isinstance(a, int):
        raise TypeError("a must be int")
    # 多くの出題では a > 2 が暗黙の前提（a=1,2 は正の三辺にならない）
    if a <= 2:
        raise ValueError("a must be greater than 2")


def _main_algorithm(a: int) -> List[int]:
    # a が奇数: (a, (a^2-1)/2, (a^2+1)/2)
    if a & 1:
        aa = a * a
        b = (aa - 1) // 2
        c = b + 1  # (aa + 1)//2 と同値
        return [a, b, c]

    # a が偶数: a=2k => (a, k^2-1, k^2+1)
    k = a // 2
    kk = k * k
    b = kk - 1
    c = kk + 1
    return [a, b, c]


def pythagoreanTriple(a: int) -> List[int]:
    # HackerRank 提出用：I/O なしで関数のみ完成させる
    # 競プロ想定の最短ルートを利用
    return solve_competitive(a)


if __name__ == '__main__':
    fptr = open(os.environ['OUTPUT_PATH'], 'w')

    a = int(input().strip())

    triple = pythagoreanTriple(a)

    fptr.write(' '.join(map(str, triple)))
    fptr.write('\n')

    fptr.close()
```


# Pythagorean Triple - 与えられた一辺から整数解を O(1) で構成

## Table of Contents

* [概要](#overview)
* [アルゴリズム要点（TL;DR）](#tldr)
* [図解](#figures)
* [正しさのスケッチ](#correctness)
* [計算量](#complexity)
* [Python 実装](#impl)
* [CPython最適化ポイント](#cpython)
* [エッジケースと検証観点](#edgecases)
* [FAQ](#faq)

<h2 id="overview">概要</h2>

直角三角形の直角をはさむ辺（cathetus）の一つ `a` が与えられるとき、**整数のピタゴラス数** `(a, b, c)`（`a^2 + b^2 = c^2`）を 1 つ構成して出力します。
解は複数あり得ますが **どれか 1 つ**でよい、という条件により、探索ではなく **閉形式（公式）**で確実に構成できます。

前提:

* `a` は正の整数
* 多くの設定で `a > 2` が暗黙（`a = 1, 2` では正の整数解を作れません）

<h2 id="tldr">アルゴリズム要点（TL;DR）</h2>

* 戦略: `a` の **偶奇**で既知の生成式を適用して `(a, b, c)` を一発生成
* データ構造: 不要（整数演算のみ）
* 生成式:

  * `a` が奇数:

    * `b = (a^2 - 1) // 2`
    * `c = (a^2 + 1) // 2`
  * `a` が偶数（`a = 2k`）:

    * `b = k^2 - 1`
    * `c = k^2 + 1`
* 計算量:

  * Time: **O(1)**
  * Space: **O(1)**

<h2 id="figures">図解</h2>

### フローチャート（構成手順）

```mermaid
flowchart TD
  Start[Start solve] --> ReadA[Read a]
  ReadA --> CheckOdd{a is odd}
  CheckOdd -- Yes --> SqOdd[Compute aa as a times a]
  SqOdd --> BOdd[Compute b as aa minus 1 over 2]
  BOdd --> COdd[Compute c as b plus 1]
  COdd --> OutOdd[Return a b c]
  CheckOdd -- No --> Half[Compute k as a over 2]
  Half --> SqEven[Compute kk as k times k]
  SqEven --> BEven[Compute b as kk minus 1]
  BEven --> CEven[Compute c as kk plus 1]
  CEven --> OutEven[Return a b c]
```

上図の通り、分岐は偶奇判定のみで、以降は乗算と整数除算で完結します。

### データフロー図（入力から出力まで）

```mermaid
graph LR
  A[Input a] --> B[Parity check]
  B --> C[Formula branch]
  C --> D[Integer arithmetic]
  D --> E[Output triple]
```

入力 `a` は偶奇判定に入り、対応する公式で `b, c` を生成して出力します。探索やループは発生しません。

<h2 id="correctness">正しさのスケッチ</h2>

**目的**: 返す `[a, b, c]` が `a^2 + b^2 = c^2` を満たし、`b, c` が正の整数であること。

### ケース1: `a` が奇数

* `b = (a^2 - 1) / 2`, `c = (a^2 + 1) / 2` は `a` が奇数なら `a^2 - 1` は偶数なので整数。
* 差と和の関係:

  * `c - b = 1`
  * `c + b = a^2`
* よって

  * `c^2 - b^2 = (c - b)(c + b) = 1 * a^2 = a^2`
  * したがって `a^2 + b^2 = c^2` が成立。
* `a > 2` なら `b >= 1`, `c > b` で正の整数。

### ケース2: `a` が偶数（`a = 2k`）

* `b = k^2 - 1`, `c = k^2 + 1` は整数。
* `c^2 - b^2 = (c - b)(c + b) = 2 * (2k^2) = 4k^2 = a^2`
* よって `a^2 + b^2 = c^2` が成立。
* `a > 2` なら `k >= 2` なので `b = k^2 - 1 >= 3` で正の整数。

**終了性**: 分岐 1 回・定数回の算術で終了します。

<h2 id="complexity">計算量</h2>

* Time: **O(1)**
* Space: **O(1)**

探索型（`b` を走査して平方判定する等）と比較して、入力が大きい場合でも実行時間が増えません。

<h2 id="impl">Python 実装</h2>

以下は **LeetCode class 形式**での実装です（純粋関数的、型注釈は pylance 互換）。
注: HackerRank の提出関数は `pythagoreanTriple(a)` でしたが、ここでは解説用に LeetCode 形式 `Solution.pythagoreanTriple` を示します。

```python
from __future__ import annotations

from typing import List


class Solution:
    def pythagoreanTriple(self, a: int) -> List[int]:
        """
        Construct any Pythagorean triple [a, b, c] such that a^2 + b^2 = c^2.

        Args:
            a: length of one cathetus (positive integer)
        Returns:
            [a, b, c] in any order
        Raises:
            ValueError: if a <= 2 (no positive-integer triple exists for a = 1 or 2)
        """
        # 入力制約が明示されていない場合でも、a=1,2 は正の整数解が作れないため防御
        if a <= 2:
            raise ValueError("a must be greater than 2")

        # a が奇数なら: b=(a^2-1)/2, c=(a^2+1)/2
        if a & 1:
            aa = a * a
            b = (aa - 1) // 2
            c = b + 1
            return [a, b, c]

        # a が偶数なら: a=2k, b=k^2-1, c=k^2+1
        k = a // 2
        kk = k * k
        b = kk - 1
        c = kk + 1
        return [a, b, c]
```

<h2 id="cpython">CPython最適化ポイント</h2>

* **探索をしない**: ループや `math.isqrt` などの平方判定を排し、算術だけにすると CPython で最速。
* 乗算回数を抑える:

  * `aa = a * a`、`kk = k * k` のように中間値を保持し、同じ計算を繰り返さない。
* 分岐は `a & 1`:

  * `% 2` よりもビット演算が軽量になりやすい（差は小さいが定数最適化として妥当）。
* Python の整数は任意精度:

  * オーバーフローを気にせず `a*a` を使える（ただし巨大入力では桁数に比例して乗算コストは増える点は理解しておく）。

<h2 id="edgecases">エッジケースと検証観点</h2>

* `a = 1`:

  * 正の整数 `b, c` を満たす解は存在しない（例外扱いが妥当）
* `a = 2`:

  * 同様に正の整数解は存在しない
* `a = 3`（奇数の最小クラス）:

  * `3 4 5` が生成される
* `a = 4`（偶数の最小クラス）:

  * `4 3 5` が生成される
* `a` が非常に大きい:

  * O(1) 手順だが、多倍長整数の乗算コストは桁数に依存するため、入出力のボトルネックも含めて挙動確認

<h2 id="faq">FAQ</h2>

**Q1. なぜ偶奇で式が違うのですか？**
A. 既知のピタゴラス数の生成法を `a` 固定に合わせて使い分けています。奇数のときは `(a^2 ± 1)/2` が整数になり、偶数のときは `a=2k` と置くと簡潔な形で整数解が作れます。

**Q2. 出力の順序は `a b c` でないといけませんか？**
A. 問題文が「任意順でよい」としている場合は問題ありません。要求が厳密なら `a b c` の順にそろえて出力します。

**Q3. 常に `b` は正ですか？**
A. `a > 2` を前提にすれば、奇数・偶数のどちらの式でも `b` は正になります。`a=1,2` だけは `b` が 0 以下になり得るため、除外（例外）するのが安全です。

**Q4. 原始ピタゴラス数（互いに素）になりますか？**
A. 必ずしもそうとは限りません。問題は「どれか 1 解」でよいので、原始性は要求されません。必要なら追加条件（`gcd(a,b,c)=1` など）に応じて生成法を変えます。
